In [1]:
import pyspark.ml.feature as ft
from pyspark.ml import Pipeline
import pyspark.ml.evaluation as ev
import pyspark.sql.types as typ

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from mmlspark.lightgbm import LightGBMRegressor
from pyspark.ml.evaluation import RegressionEvaluator


# 如果你是通過spark-submit來運行，則需要先實例化一個spark session對象， 在pyspark中spark session對像已經默認生成
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
sc = SparkContext
spark = SparkSession.builder\
            .appName("LightGBM")\
            .config('spark.executor.instances','2')\
            .config("spark.executor.memory", '4g')\
            .config("spark.debug.maxToStringFields", "100")\
            .master("yarn")\
            .getOrCreate()
cs = spark.sparkContext
# spark.sparkContext._conf.getAll() # 可看所有資訊
# .config("spark.driver.memory", '8g')\
# .config("spark.executor.memory", '4g')\

In [2]:
# function
def info(slef):
    for i in spark.sparkContext._conf.getAll():
        if i[0] in ['spark.executor.instances','spark.executor.cores','spark.executor.memory']:
            print(f'{i[0]} : {i[1]}')
    print(f'Partitions : {app_train.rdd.getNumPartitions()}')

In [3]:
# 讀取csv數據，這裡讀取的是事先使用hadoop fs -put命令上傳到hadoop裡的數據
final_df = spark.read.csv("/data/application_train_cleaning_v2.csv", header='true', inferSchema='true')
# final_df = spark.read.parquet("/data/BDSE12_03G_HomeCredit_V2.csv_parquet")
final_df = final_df.repartition(4)
# final_df = final_df.drop(*['_c0',
#                              'PREV_NFLAG_INSURED_ON_APPROVAL_0.0_rate',
#                             'PREV_NFLAG_INSURED_ON_APPROVAL_1.0_rate',
#                             'PREV_NFLAG_INSURED_ON_APPROVAL_nan_rate'])
app_train = final_df[final_df['TARGET'].isNotNull()]
app_test = final_df[final_df['TARGET'].isNull()]

In [4]:
# spark機器學習要求輸入的DataFrame類型為數值類型， 將本來的string欄位轉換成double，並替代空值
for col, t in app_train.dtypes:
    if t == "string":
        app_train = app_train.withColumn(col, app_train[col].cast("double"))

app_train = app_train.withColumn("TARGET", app_train["TARGET"].cast("int"))
app_train = app_train.fillna(999999)

In [5]:
# 跟在普通單機上做訓練時不同，spark做訓練時所有特徵列需要通過VectorAssembler轉換成特徵矩陣，才能用來訓練
featuresCreator = ft.VectorAssembler(
    inputCols=[col for col in app_train.columns[:] if col not in ["TARGET",'SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']],
    outputCol='features'
    )

In [6]:
%%time
# 實例化一個LightGBM Regressor， 其參數和單機版本類似但不盡相同， 文檔可以在以下鏈接找到：
# https://mmlspark.azureedge.net/docs/pyspark/LightGBMRegressor.html
lgbm = LightGBMRegressor(
    boostingType="goss",
    numIterations=120,
    objective='binary',
    learningRate=0.1,
    baggingSeed=50,
    lambdaL1=0.4,
    lambdaL2=0.4,
    baggingFraction=0.87,
    minSumHessianInLeaf=0.003,
    maxDepth=8,
    featureFraction=0.66,
    numLeaves=20,
    labelCol="TARGET"
                          )
# 建立一個pipeline，簡化訓練步驟
pipeline = Pipeline(stages=[
                # 特徵整理
                featuresCreator,
                # 模型名稱
                    lgbm])
# 這裡是將數據分成訓練集和驗證集，測試模型預測效果
train_df, test_df = app_train.randomSplit([0.7, 0.3])

vmodel = pipeline.fit(train_df)
t_prediction = vmodel.transform(test_df)
evaluator = ev.BinaryClassificationEvaluator(
     rawPredictionCol='prediction',
     labelCol='TARGET')

print(info(t))
print('')
print(f'AUC : {evaluator.evaluate(t_prediction)}')


Py4JJavaError: An error occurred while calling o121.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 4 times, most recent failure: Lost task 0.3 in stage 4.0 (TID 18, bdse209.example.org, executor 2): java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:607)
	at java.net.Socket.connect(Socket.java:556)
	at java.net.Socket.<init>(Socket.java:452)
	at java.net.Socket.<init>(Socket.java:229)
	at com.microsoft.ml.spark.lightgbm.TrainUtils$.getNetworkInitNodes(TrainUtils.scala:299)
	at com.microsoft.ml.spark.lightgbm.TrainUt

In [ ]:
%%time

# 交叉驗證

paramGrid = ParamGridBuilder()\
    .addGrid(lgbm.learningRate, [0.095,0.1,0.105])\
    .build()

cv = CrossValidator(
    estimator=lgbm,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds = 5)

cv_pipeline = Pipeline(stages=[featuresCreator,cv])

cv_model = cv_pipeline.fit(train_df)
cv_prediction = cv_model.transform(test_df)

print(f'Full AUC {evaluator.evaluate(cv_prediction)}')

In [ ]:
%%time

# 找出最佳模型

from pyspark.ml.tuning import TrainValidationSplit
paramGrid = ParamGridBuilder()\
    .addGrid(lgbm.numLeaves, [10,20,30])\
    .addGrid(lgbm.numIterations, [100,160,200])\
    .addGrid(lgbm.baggingSeed, [25,50,75])\
    .build()
tvs = TrainValidationSplit( estimator=lgbm,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    trainRatio = 0.8)
# 最佳模型
tvs_pipeline = Pipeline(stages=[featuresCreator,tvs])

tvs_pipelineModel = tvs_pipeline.fit(train_df)

prediction = tvs_pipelineModel.transform(test_df)
print(f'Full AUC {evaluator.evaluate(prediction)}')


In [ ]:
%%time

# 找出最佳模型+交叉驗證

from pyspark.ml.tuning import TrainValidationSplit
paramGrid = ParamGridBuilder()\
    .addGrid(lgbm.numLeaves, [10,20,30])\
    .addGrid(lgbm.numIterations, [100,200,300])\
    .addGrid(lgbm.baggingSeed, [25,50,75])\
    .build()

cv = CrossValidator(
    estimator=lgbm,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds = 10)

tvs_pipeline = Pipeline(stages=[featuresCreator,cv])

tvs_pipelineModel = tvs_pipeline.fit(train_df)

prediction = tvs_pipelineModel.transform(test_df)
print(f'Full AUC {evaluator.evaluate(prediction)}')
